In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import numpy as np
from AHRS import AHRS
import os
import csv
from quaternions import quaternRotate
from funcions import convertData, convertDataHex, grafica, find_discontinuity, obtenir_x_valors
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import random
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

# **CREACIÓ DE FITXERS**
S'han creat diversos fitxers per ajudar a la manipulació de les dades i poder dur a terme els càlculs realitzats en el fitxer ******.ipynb

In [2]:
# directory = "C:/Users/Maria/OneDrive/Escritorio/4tMatCAD/TFG/Dades_posturograf_IMU/Mataro_postu_IMU/Mataro_posturograf"
directory = "C:/Users/Maria/OneDrive/Escritorio/4tMatCAD/TFG/TFG Maria/Mataro_posturograf"

In [3]:
def write_csv(data, filename, column_names):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)  # Escribir los nombres de las columnas
        row = []
        for item in data:
            if item == "HOLA":
                writer.writerow(row)
                row = []
            else:
                row.append(item)
        if row:  # Escribir la última fila si quedan elementos en 'row'
            writer.writerow(row)

### **VAL_ROA.csv**
Aquest fitxer ens facilita el càlcul de la predicció de les valoracions de la prova ROA a partir de la seva fòrmula. Consta de 6 columnes:
- **Edad:** La edat del pacient
- **Val_ROA:** La valoració de la prova ROA segons calculat per l'aplicació 
- **Despl_ML1:** Desplaçament en direcció *mediolateral* de la primera prova de ROA
- **Despl_AP1:** Desplaçament en direcció *anteroposterior* de la primera prova de ROA
- **Despl_ML2:** Desplaçament en direcció *mediolateral* de la segona prova de ROA
- **Despl_AP2:** Desplaçament en direcció *anteroposterior* de la segona prova de ROA

In [4]:
list= []
file_count= 0
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        file_count += 1

        if file_count > 107 and file_count < 112:
            continue
        df_dades = pd.read_csv(os.path.join(directory, filename), delimiter="\t", encoding="ansi", skiprows=1, chunksize=11, header=None).get_chunk()
        # Obté l'edat del pacient
        age = df_dades[9][0]
        list.append(age)
        list.append(df_dades[[17]].values[0][0]) # val_roa

        # Obté els índexs de les files que compleixen la condició
        mask = df_dades.iloc[:, 1] == 'ROA'
        roa_indices = mask[mask].index
        if len(roa_indices)==2: # Nomes pels pacients que tenen 2 proves roa
            for index in roa_indices:
                # if(df_dades[9][index]==)
                list.append(df_dades[9][index]) # despl. ml
                
                list.append(df_dades[10][index]) # despl. ap
        list.append("HOLA")

In [5]:
column_names = ['Edad', 'Val_ROA', 'Despl_ML1', 'Despl_AP1', 'Despl_ML2', 'Despl_AP2']
write_csv(list, 'VAL_ROA.csv', column_names)

### **VAL_ROC.csv**
Aquest fitxer ens facilita el càlcul de la predicció de les valoracions de la prova ROC a partir de la seva fòrmula. Consta de 6 columnes:
- **Edad:** La edat del pacient
- **Val_ROA:** La valoració de la prova ROC segons calculat per l'aplicació 
- **Despl_ML1:** Desplaçament en direcció *mediolateral* de la primera prova de ROC
- **Despl_AP1:** Desplaçament en direcció *anteroposterior* de la primera prova de ROC
- **Despl_ML2:** Desplaçament en direcció *mediolateral* de la segona prova de ROC
- **Despl_AP2:** Desplaçament en direcció *anteroposterior* de la segona prova de ROC

In [6]:
list= []
file_count= 0
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        file_count += 1

        if file_count > 107 and file_count < 112:
            continue
        df_dades = pd.read_csv(os.path.join(directory, filename), delimiter="\t", encoding="ansi", skiprows=1, chunksize=11, header=None).get_chunk()
        # Obté l'edat del pacient
        age = df_dades[9][0]
        list.append(age)
        list.append(df_dades[[22]].values[0][0]) # val_roc

        # Obté els índexs de les files que compleixen la condició
        mask = df_dades.iloc[:, 1] == 'ROC'
        roa_indices = mask[mask].index
        if len(roa_indices)==2: # Nomes pels pacients que tenen 2 proves roa
            for index in roa_indices:
                list.append(df_dades[9][index]) # despl. ml
                list.append(df_dades[10][index]) # despl. ap
        list.append("HOLA")

In [7]:
column_names = ['Edad', 'Val_ROC', 'Despl_ML1', 'Despl_AP1', 'Despl_ML2', 'Despl_AP2']
write_csv(list, 'VAL_ROC.csv', column_names)

### **VAL_RGA.csv**
Aquest fitxer ens facilita el càlcul de la predicció de les valoracions de la prova RGA a partir de la seva fòrmula. Consta de 6 columnes:
- **Edad:** La edat del pacient
- **Val_ROA:** La valoració de la prova RGA segons calculat per l'aplicació 
- **Despl_ML1:** Desplaçament en direcció *mediolateral* de la primera prova de RGA
- **Despl_AP1:** Desplaçament en direcció *anteroposterior* de la primera prova de RGA
- **Despl_ML2:** Desplaçament en direcció *mediolateral* de la segona prova de RGA
- **Despl_AP2:** Desplaçament en direcció *anteroposterior* de la segona prova de RGA

In [8]:
list= []
file_count= 0
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        file_count += 1

        if file_count > 107 and file_count < 112:
            continue
        df_dades = pd.read_csv(os.path.join(directory, filename), delimiter="\t", encoding="ansi", skiprows=1, chunksize=11, header=None).get_chunk()
        # Obté l'edat del pacient
        age = df_dades[9][0]
        list.append(age)
        list.append(df_dades[[27]].values[0][0]) # val_rga

        # Obté els índexs de les files que compleixen la condició
        mask = df_dades.iloc[:, 1] == 'RGA'
        roa_indices = mask[mask].index
        if len(roa_indices)==2: # Nomes pels pacients que tenen 2 proves roa
            for index in roa_indices:
                list.append(df_dades[9][index]) # despl. ml
                list.append(df_dades[10][index]) # despl. ap
        list.append("HOLA")

In [9]:
column_names = ['Edad', 'Val_RGA', 'Despl_ML1', 'Despl_AP1', 'Despl_ML2', 'Despl_AP2']
write_csv(list, 'VAL_RGA.csv', column_names)

### **VAL_RGC.csv**
Aquest fitxer ens facilita el càlcul de la predicció de les valoracions de la prova RGC a partir de la seva fòrmula. Consta de 6 columnes:
- **Edad:** La edat del pacient
- **Val_ROA:** La valoració de la prova RGC segons calculat per l'aplicació 
- **Despl_ML1:** Desplaçament en direcció *mediolateral* de la primera prova de RGC
- **Despl_AP1:** Desplaçament en direcció *anteroposterior* de la primera prova de RGC
- **Despl_ML2:** Desplaçament en direcció *mediolateral* de la segona prova de RGC
- **Despl_AP2:** Desplaçament en direcció *anteroposterior* de la segona prova de RGC

In [10]:
list= []
file_count= 0
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        file_count += 1

        if file_count > 107 and file_count < 112:
            continue
        df_dades = pd.read_csv(os.path.join(directory, filename), delimiter="\t", encoding="ansi", skiprows=1, chunksize=11, header=None).get_chunk()
        # Obté l'edat del pacient
        age = df_dades[9][0]
        list.append(age)
        list.append(df_dades[[32]].values[0][0]) # val_rgc

        # Obté els índexs de les files que compleixen la condició
        mask = df_dades.iloc[:, 1] == 'RGC'
        roa_indices = mask[mask].index
        if len(roa_indices)==2: # Nomes pels pacients que tenen 2 proves roa
            for index in roa_indices:
                list.append(df_dades[9][index]) # despl. ml
                list.append(df_dades[10][index]) # despl. ap
        list.append("HOLA")

In [11]:
column_names = ['Edad', 'Val_RGC', 'Despl_ML1', 'Despl_AP1', 'Despl_ML2', 'Despl_AP2']
write_csv(list, 'VAL_RGC.csv', column_names)

### **indices.csv**
Aquest fitxer ens facilita el càlcul de les prediccions dels índexos sensorials (SOM, VIS, VEST i DIN). Consta de 11 columnes on guardem l'historial del pacient, els valors a partir dels quals es calculen els indexos i els indexos originals calculats per l'aplicació:
- **Num_Historial:** El número identificatiu de cada pacient 
- **Val_ROA:** Índex de valoració de la prova ROA calculada per l'aplicació
- **Val_ROC:** Índex de valoració  de la prova ROC calculada per l'aplicació
- **Val_RGA:** Índex de valoració  de la prova RGA calculada per l'aplicació
- **Val_RGC:** Índex de valoració  de la prova RGC calculada per l'aplicació
- **Val_MARCHA:** Índex funcional de valoració de la marcha humana que quantifica paràmetres significatius que diferencien població normal de patològica
- **Indice_VIS:** Índex que indica el percentatge de valoració del sistema Visual calculat per l'aplicació
- **Indice_VEST:** Índex que indica el percentatge de valoració del sistema Vestibular calculat per l'aplicació
- **Indice_SOM:** Índex que indica el percentatge de valoració del sistema Somatosensorial calculat per l'aplicació
- **Indice_DIN:** Índex que indica el percentatge de valoració del sistema Dinàmic calculat per l'aplicació

In [12]:
list= []
file_count= 0
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        file_count += 1

        if file_count > 107 and file_count < 112:
            continue
        df_dades = pd.read_csv(os.path.join(directory, filename), delimiter="\t", encoding="ansi", skiprows=1, chunksize=11, header=None).get_chunk()
        list.append(df_dades[[0]].values[0][0]) # Num_Historial
        list.append(df_dades[[9]].values[0][0]) # Edad

        list.append(df_dades[[17]].values[0][0]) # val_roa
        list.append(df_dades[[22]].values[0][0]) # val_roc
        list.append(df_dades[[27]].values[0][0]) # val_rga
        list.append(df_dades[[32]].values[0][0]) # val_rgc
        list.append(df_dades[[47]].values[0][0]) # val_marcha
        
        list.append(df_dades[53][0]) # Indice_SOM
        list.append(df_dades[54][0]) # Indice_VIS
        list.append(df_dades[55][0]) # Indice_VEST
        list.append(df_dades[56][0]) # Indice_DOM
        list.append(df_dades[57][0]) # Indice_DIN

        list.append("HOLA")

In [13]:
column_names = ['Num_Historial', 'Edat', 'Val_ROA', 'Val_ROC', 'Val_RGA', 'Val_RGC','Val_MARCHA', 'Indice_SOM', 'Indice_VIS', 'Indice_VEST', 'Indice_DOM', 'Indice_DIN']
write_csv(list, 'indices.csv', column_names)

### **regressio_ROA.csv**
Aquest fitxer ens facilita el càlcul de la predicció de l'edat del pacient mitjançant una regressió lineal multiclasse. El fitxer consta de 12 paràmetres: L'historial i l'edat del pacient i la mitjana dels valors calculats per l'aplicació en la prova ROA:
- **Desplaz.Total(mm):** Distància a l'origen de la plataforma, expressada en mil·límetres, del centre de pressions del núvol de punts (vector desplaçament) descrit per la projecció del cdg	
- **Angulo Desplaz.(º):** Orientació del vector desplaçament, expressat en graus.
- **Dispers ML (mm):** Dispersió, expressada en mil·límetres, del núvol de punts descrit per la projecció del cdg en direcció ML.
- **Dispers AP (mm):** Dispersió, expressada en mil·límetres, del núvol de punts descrit per la projecció del cdg en direcció AP.
- **Area barrida (mm2):** Estimador de l'àrea aproximada, expressada en mil·límetres quadrats, en què es produeix el balanceig del subjecte, calculada a partir de l'instant en què s'inicia la prova fins que s'acaba.
- **Velocidad media (m/s):** Estimador de la velocitat mitjana, expressat en metres per segon, de la projecció del cdg del subjecte durant la prova, calculada a partir de l'instant en què s'inicia la prova fins que s'acaba.
- **Desplazam.AP(mm):** Distància màxima, expressada en mil·límetres, en direcció ML i AP assolida durant la prova. 
- **Fuerza Max ML (N):** Força màxima, expressada a Newtons, en direcció ML exercida durant la prova.
- **Fuerza Max AP (N):** Força màxima, expressada a Newtons, en direcció AP exercida durant la prova.

In [14]:
dades_llista_ROA = pd.DataFrame()

for fitxer_nom in os.listdir(directory):
    if fitxer_nom.endswith('.txt'):
        fitxer_ruta = os.path.join(directory, fitxer_nom)
        df_resultats = pd.read_csv(fitxer_ruta, delimiter="\t", encoding="cp1252", skiprows=2, chunksize=9,index_col=False, on_bad_lines='skip').get_chunk()

        # Mitjana de cada valor per la prov ROA
        mask = df_resultats.iloc[:, 1] == 'ROA'
        roa_indices = mask[mask].index
        if len(roa_indices) == 2:  # Només pels pacients que tenen 2 proves ROA
            rows = df_resultats.iloc[roa_indices, 3:13]
            rows = rows.apply(pd.to_numeric, errors='coerce')
            mean = rows.mean().reset_index(drop=True)
            df_mean = pd.DataFrame([mean])

            # L'edat del pacient
            df_dades = pd.read_csv(fitxer_ruta, delimiter="\t", encoding="ansi", skiprows=1, chunksize=1, header=None).get_chunk()
            edat = df_dades.iloc[0, 9:10]
            edat = edat.rename({9: 'Edat'})
            edat = pd.DataFrame([edat])

            # El num d'historial del pacient
            df_historial = df_dades.iloc[0, 0:1]
            df_historial = df_historial.rename({0: 'Num Historial'})
            df_historial = pd.DataFrame([df_historial])

            # Concatenate the necessary columns
            dades_llista_aux = pd.concat([df_historial, df_mean, edat], axis=1)

            # Append to the final DataFrame
            dades_llista_ROA = pd.concat([dades_llista_ROA, dades_llista_aux], ignore_index=True)

dades_llista_ROA.columns = ['Num Historial',	'Desplaz.Total(mm)', 'Angulo Desplaz.','Dispers ML (mm)','Dispers AP (mm)','Area barrida (mm2)', 'Velocidad media (m/s)', 'Desplazam.ML (mm)','Desplazam.AP(mm)','Fuerza Max ML (N)','Fuerza Max AP (N)', 'Edat']
dades_llista_ROA.to_csv('Regressio_ROA.csv', index=False)

### **regressio_ROC.csv**
Aquest fitxer ens facilita el càlcul de la predicció de l'edat del pacient mitjançant una regressió lineal multiclasse. El fitxer consta de 12 paràmetres: L'historial i l'edat del pacient i la mitjana dels valors calculats per l'aplicació en la prova ROC:
- **Desplaz.Total(mm):** Distància a l'origen de la plataforma, expressada en mil·límetres, del centre de pressions del núvol de punts (vector desplaçament) descrit per la projecció del cdg	
- **Angulo Desplaz.(º):** Orientació del vector desplaçament, expressat en graus.
- **Dispers ML (mm):** Dispersió, expressada en mil·límetres, del núvol de punts descrit per la projecció del cdg en direcció ML.
- **Dispers AP (mm):** Dispersió, expressada en mil·límetres, del núvol de punts descrit per la projecció del cdg en direcció AP.
- **Area barrida (mm2):** Estimador de l'àrea aproximada, expressada en mil·límetres quadrats, en què es produeix el balanceig del subjecte, calculada a partir de l'instant en què s'inicia la prova fins que s'acaba.
- **Velocidad media (m/s):** Estimador de la velocitat mitjana, expressat en metres per segon, de la projecció del cdg del subjecte durant la prova, calculada a partir de l'instant en què s'inicia la prova fins que s'acaba.
- **Desplazam.AP(mm):** Distància màxima, expressada en mil·límetres, en direcció ML i AP assolida durant la prova. 
- **Fuerza Max ML (N):** Força màxima, expressada a Newtons, en direcció ML exercida durant la prova.
- **Fuerza Max AP (N):** Força màxima, expressada a Newtons, en direcció AP exercida durant la prova.

In [15]:
dades_llista_ROC = pd.DataFrame()

for fitxer_nom in os.listdir(directory):
    if fitxer_nom.endswith('.txt'):
        fitxer_ruta = os.path.join(directory, fitxer_nom)
        df_resultats = pd.read_csv(fitxer_ruta, delimiter="\t", encoding="cp1252", skiprows=2, chunksize=9,index_col=False, on_bad_lines='skip').get_chunk()

        # Mitjana de cada valor per la prov ROC
        mask = df_resultats.iloc[:, 1] == 'ROC'
        roc_indices = mask[mask].index
        if len(roc_indices) == 2:  # Només pels pacients que tenen 2 proves ROC
            rows = df_resultats.iloc[roc_indices, 3:13]
            rows = rows.apply(pd.to_numeric, errors='coerce')
            mean = rows.mean().reset_index(drop=True)
            df_mean = pd.DataFrame([mean])

            # L'edat del pacient
            df_dades = pd.read_csv(fitxer_ruta, delimiter="\t", encoding="ansi", skiprows=1, chunksize=1, header=None).get_chunk()
            edat = df_dades.iloc[0, 9:10]
            edat = edat.rename({9: 'Edat'})
            edat = pd.DataFrame([edat])

            # El num d'historial del pacient
            df_historial = df_dades.iloc[0, 0:1]
            df_historial = df_historial.rename({0: 'Num Historial'})
            df_historial = pd.DataFrame([df_historial])

            # Concatenate the necessary columns
            dades_llista_aux = pd.concat([df_historial, df_mean, edat], axis=1)

            # Append to the final DataFrame
            dades_llista_ROC = pd.concat([dades_llista_ROC, dades_llista_aux], ignore_index=True)

dades_llista_ROC.columns = ['Num Historial',	'Desplaz.Total(mm)', 'Angulo Desplaz.','Dispers ML (mm)','Dispers AP (mm)','Area barrida (mm2)', 'Velocidad media (m/s)', 'Desplazam.ML (mm)','Desplazam.AP(mm)','Fuerza Max ML (N)','Fuerza Max AP (N)', 'Edat']
dades_llista_ROC.to_csv('Regressio_ROC.csv', index=False)

### **regressio_RGA.csv**
Aquest fitxer ens facilita el càlcul de la predicció de l'edat del pacient mitjançant una regressió lineal multiclasse. El fitxer consta de 12 paràmetres: L'historial i l'edat del pacient i la mitjana dels valors calculats per l'aplicació en la prova RGA:
- **Desplaz.Total(mm):** Distància a l'origen de la plataforma, expressada en mil·límetres, del centre de pressions del núvol de punts (vector desplaçament) descrit per la projecció del cdg	
- **Angulo Desplaz.(º):** Orientació del vector desplaçament, expressat en graus.
- **Dispers ML (mm):** Dispersió, expressada en mil·límetres, del núvol de punts descrit per la projecció del cdg en direcció ML.
- **Dispers AP (mm):** Dispersió, expressada en mil·límetres, del núvol de punts descrit per la projecció del cdg en direcció AP.
- **Area barrida (mm2):** Estimador de l'àrea aproximada, expressada en mil·límetres quadrats, en què es produeix el balanceig del subjecte, calculada a partir de l'instant en què s'inicia la prova fins que s'acaba.
- **Velocidad media (m/s):** Estimador de la velocitat mitjana, expressat en metres per segon, de la projecció del cdg del subjecte durant la prova, calculada a partir de l'instant en què s'inicia la prova fins que s'acaba.
- **Desplazam.AP(mm):** Distància màxima, expressada en mil·límetres, en direcció ML i AP assolida durant la prova. 
- **Fuerza Max ML (N):** Força màxima, expressada a Newtons, en direcció ML exercida durant la prova.
- **Fuerza Max AP (N):** Força màxima, expressada a Newtons, en direcció AP exercida durant la prova.

In [16]:
dades_llista_RGA = pd.DataFrame()

for fitxer_nom in os.listdir(directory):
    if fitxer_nom.endswith('.txt'):
        fitxer_ruta = os.path.join(directory, fitxer_nom)
        df_resultats = pd.read_csv(fitxer_ruta, delimiter="\t", encoding="cp1252", skiprows=2, chunksize=9,index_col=False, on_bad_lines='skip').get_chunk()

        # Mitjana de cada valor per la prov RGA
        mask = df_resultats.iloc[:, 1] == 'RGA'
        rga_indices = mask[mask].index
        if len(rga_indices) == 2:  # Només pels pacients que tenen 2 proves RGA
            rows = df_resultats.iloc[rga_indices, 3:13]
            rows = rows.apply(pd.to_numeric, errors='coerce')
            mean = rows.mean().reset_index(drop=True)
            df_mean = pd.DataFrame([mean])

            # L'edat del pacient
            df_dades = pd.read_csv(fitxer_ruta, delimiter="\t", encoding="ansi", skiprows=1, chunksize=1, header=None).get_chunk()
            edat = df_dades.iloc[0, 9:10]
            edat = edat.rename({9: 'Edat'})
            edat = pd.DataFrame([edat])

            # El num d'historial del pacient
            df_historial = df_dades.iloc[0, 0:1]
            df_historial = df_historial.rename({0: 'Num Historial'})
            df_historial = pd.DataFrame([df_historial])

            # Concatenate the necessary columns
            dades_llista_aux = pd.concat([df_historial, df_mean, edat], axis=1)

            # Append to the final DataFrame
            dades_llista_RGA = pd.concat([dades_llista_RGA, dades_llista_aux], ignore_index=True)

dades_llista_RGA.columns = ['Num Historial',	'Desplaz.Total(mm)', 'Angulo Desplaz.','Dispers ML (mm)','Dispers AP (mm)','Area barrida (mm2)', 'Velocidad media (m/s)', 'Desplazam.ML (mm)','Desplazam.AP(mm)','Fuerza Max ML (N)','Fuerza Max AP (N)', 'Edat']
dades_llista_RGA.to_csv('Regressio_RGA.csv', index=False)

### **regressio_RGC.csv**
Aquest fitxer ens facilita el càlcul de la predicció de l'edat del pacient mitjançant una regressió lineal multiclasse. El fitxer consta de 12 paràmetres: L'historial i l'edat del pacient i la mitjana dels valors calculats per l'aplicació en la prova RGC.
- **Desplaz.Total(mm):** Distància a l'origen de la plataforma, expressada en mil·límetres, del centre de pressions del núvol de punts (vector desplaçament) descrit per la projecció del cdg	
- **Angulo Desplaz.(º):** Orientació del vector desplaçament, expressat en graus.
- **Dispers ML (mm):** Dispersió, expressada en mil·límetres, del núvol de punts descrit per la projecció del cdg en direcció ML.
- **Dispers AP (mm):** Dispersió, expressada en mil·límetres, del núvol de punts descrit per la projecció del cdg en direcció AP.
- **Area barrida (mm2):** Estimador de l'àrea aproximada, expressada en mil·límetres quadrats, en què es produeix el balanceig del subjecte, calculada a partir de l'instant en què s'inicia la prova fins que s'acaba.
- **Velocidad media (m/s):** Estimador de la velocitat mitjana, expressat en metres per segon, de la projecció del cdg del subjecte durant la prova, calculada a partir de l'instant en què s'inicia la prova fins que s'acaba.
- **Desplazam.AP(mm):** Distància màxima, expressada en mil·límetres, en direcció ML i AP assolida durant la prova. 
- **Fuerza Max ML (N):** Força màxima, expressada a Newtons, en direcció ML exercida durant la prova.
- **Fuerza Max AP (N):** Força màxima, expressada a Newtons, en direcció AP exercida durant la prova.

In [17]:
dades_llista_RGC = pd.DataFrame()

for fitxer_nom in os.listdir(directory):
    if fitxer_nom.endswith('.txt'):
        fitxer_ruta = os.path.join(directory, fitxer_nom)
        df_resultats = pd.read_csv(fitxer_ruta, delimiter="\t", encoding="cp1252", skiprows=2, chunksize=9,index_col=False, on_bad_lines='skip').get_chunk()

        # Mitjana de cada valor per la prov RGC
        mask = df_resultats.iloc[:, 1] == 'RGC'
        rgc_indices = mask[mask].index
        if len(rgc_indices) == 2:  # Només pels pacients que tenen 2 proves RGC
            rows = df_resultats.iloc[rgc_indices, 3:13]
            rows = rows.apply(pd.to_numeric, errors='coerce')
            mean = rows.mean().reset_index(drop=True)
            df_mean = pd.DataFrame([mean])

            # L'edat del pacient
            df_dades = pd.read_csv(fitxer_ruta, delimiter="\t", encoding="ansi", skiprows=1, chunksize=1, header=None).get_chunk()
            edat = df_dades.iloc[0, 9:10]
            edat = edat.rename({9: 'Edat'})
            edat = pd.DataFrame([edat])

            # El num d'historial del pacient
            df_historial = df_dades.iloc[0, 0:1]
            df_historial = df_historial.rename({0: 'Num Historial'})
            df_historial = pd.DataFrame([df_historial])

            # Concatenate the necessary columns
            dades_llista_aux = pd.concat([df_historial, df_mean, edat], axis=1)

            # Append to the final DataFrame
            dades_llista_RGC = pd.concat([dades_llista_RGC, dades_llista_aux], ignore_index=True)

dades_llista_RGC.columns = ['Num Historial',	'Desplaz.Total(mm)', 'Angulo Desplaz.','Dispers ML (mm)','Dispers AP (mm)','Area barrida (mm2)', 'Velocidad media (m/s)', 'Desplazam.ML (mm)','Desplazam.AP(mm)','Fuerza Max ML (N)','Fuerza Max AP (N)', 'Edat']
dades_llista_RGC.to_csv('Regressio_RGC.csv', index=False)